In [1]:
from src.function.catalog.solr.work import DocWork
from src.schemas.catalog.bibframe.work import Work
import json

In [21]:
from src.schemas.settings import Settings
from pyfuseki import FusekiUpdate
settings = Settings()
fuseki = FusekiUpdate(settings.fuseki, 'bk') 

from pysolr import Solr
solr = Solr(f'{settings.solr}/solr/authority/', timeout=10)

In [2]:
with open('test.json') as file:
    reader = file.read()
    work = json.loads(reader)
    file.close()
work

{'identifiersLocal': '13',
 'adminMetadata': {'status': {'label': 'novo', 'value': 'n'}},
 'isPartOf': 'https://bibliokeia.com/catalog/works',
 'contribution': [{'agent': 'https://bibliokeia.com/authority/PersonalName/3',
   'label': 'Popper, Karl R.',
   'role': 'http://id.loc.gov/vocabulary/relators/aut',
   'roleLabel': 'Autor'}],
 'subject': [{'label': 'Metodologia',
   'lang': 'por',
   'uri': 'https://bibliokeia.com/authority/Topic/1',
   'type': 'Topic'}],
 'type': 'Text',
 'title': {'mainTitle': 'Conjecturas e refutaÃ§Ãµes', 'subtitle': ''},
 'content': {'label': 'Texto',
  'uri': 'http://id.loc.gov/vocabulary/contentTypes/txt'},
 'language': [{'label': 'por',
   'uri': 'http://id.loc.gov/vocabulary/languages/por'}]}

In [3]:
request = Work(**work)
request

Work(adminMetadata=AdminMetadata(assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', changeDate=None, creationDate=datetime.date(2023, 11, 30), descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', descriptionConventions='http://id.loc.gov/vocabulary/descriptionConventions/local', generationProcess='BiblioKeia v.1', generationDate='2023-11-30T15:33:27', status=Status(value='n', label='novo')), identifiersLocal='13', type='Text', content=Element(label='Texto', lang=None, uri='http://id.loc.gov/vocabulary/contentTypes/txt', type=None), language=[Element(label='por', lang=None, uri='http://id.loc.gov/vocabulary/languages/por', type=None)], title=Title(type='bf:Title', mainTitle='Conjecturas e refutaÃ§Ãµes', subtitle='', label=None), classification=None, contribution=[Contribution(agent='https://bibliokeia.com/authority/PersonalName/3', label='Popper, Karl R.', role='http://id.loc.gov/vocabul

In [5]:
contribution = request.contribution[0]

In [6]:
contribution

Contribution(agent='https://bibliokeia.com/authority/PersonalName/3', label='Popper, Karl R.', role='http://id.loc.gov/vocabulary/relators/aut', roleLabel='Autor')

In [9]:
uri = f'https://bibliokeia.com/catalog/works/{request.identifiersLocal}'
sparql = f"""PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
            INSERT DATA {{
             GRAPH <{contribution.agent}> {{
             <{contribution.agent}> bf:contributionOf <{uri}> .             
             }} }} """
print(sparql)

PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
            INSERT DATA {
             GRAPH <https://bibliokeia.com/authority/PersonalName/3> {
             <https://bibliokeia.com/authority/PersonalName/3> bf:contributionOf <https://bibliokeia.com/catalog/works/13> .             
             } } 


In [11]:
response = fuseki.run_sparql(sparql) 
response

In [13]:
contribution.agent

'https://bibliokeia.com/authority/PersonalName/3'

In [20]:
doc = {
    "id": contribution.agent.split("/")[-1],
    "contributionOf": {
        "add": {
            "id": 'testeID',
            "label": "TESTE"}
            }
}
doc

{'id': '3', 'contributionOf': {'add': {'id': 'testeID', 'label': 'TESTE'}}}

In [22]:
responseSolr =  solr.add([doc], commit=True)
responseSolr

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":282\n  }\n}'

In [24]:
request.model_dump()

{'adminMetadata': {'assigner': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'descriptionModifier': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'changeDate': None,
  'creationDate': datetime.date(2023, 11, 30),
  'descriptionLanguage': 'http://id.loc.gov/vocabulary/languages/por',
  'descriptionConventions': 'http://id.loc.gov/vocabulary/descriptionConventions/local',
  'generationProcess': 'BiblioKeia v.1',
  'generationDate': '2023-11-30T15:33:27',
  'status': {'value': 'n', 'label': 'novo'}},
 'identifiersLocal': '13',
 'type': 'Text',
 'content': {'label': 'Texto',
  'lang': None,
  'uri': 'http://id.loc.gov/vocabulary/contentTypes/txt',
  'type': None},
 'language': [{'label': 'por',
   'lang': None,
   'uri': 'http://id.loc.gov/vocabulary/languages/por',
   'type': None}],
 'title': {'type': 'bf:Title',
  'mainTitle': 'Conjecturas e refutaÃ§Ãµes',
  'subtitle': '',
  'label': None},
 'classification': None,
 'contribution': [{'agent': 'https://bibliokeia.c

In [28]:
subject = request.subject[0]
subject

Element(label='Metodologia', lang='por', uri='https://bibliokeia.com/authority/Topic/1', type='Topic')

In [29]:
subject.uri

'https://bibliokeia.com/authority/Topic/1'